In [ ]:
from language_analysis import load_latest_interaction, load_all_interactions
import math
import torch
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr
from collections import defaultdict, Counter
from egg.core.language_analysis import TopographicSimilarity

# Bee Language Analysis

In [ ]:
interaction_obj = load_latest_interaction(
    logs_root="../logs/interactions/2025-06-22",
    seed_folder="bee_default_seed42",
    split="validation",
    prefix="interaction_gpu0"
)

## Correlation analysis
Spearman test: if continuous token ranks higher on samples that truly are farther apart, ρ will be positive even without requiring them to match exactly

Accuracy: overall bearing/direction (as if drawing a straight line from nest to food)

In [ ]:
# extract tokens
vocab_size = 8
token_directions = interaction_obj.message[:, :8].argmax(dim=-1).tolist()
token_distances = interaction_obj.message[:, -1].tolist()

In [ ]:
batches = interaction_obj.aux_input["data"]
if not isinstance(batches, list):
    batches = [batches]

# get x,y coordinate
pos_list = [b.pos for b in batches]
all_pos = torch.cat(pos_list, dim=0)

# get global node indices
nest_tensor = interaction_obj.aux_input["nest_tensor"]
food_tensor = interaction_obj.aux_input["food_tensor"]

# compute Euclidean distances
real_pos_nest = all_pos[nest_tensor]
real_pos_food = all_pos[food_tensor]
real_distances = (real_pos_food - real_pos_nest).norm(dim=1).tolist()

# calculate distance token correlation
rho_dist, p_dist = spearmanr(real_distances, token_distances)
print(f"Distance token vs. real displacement: Spearman rho = {rho_dist:.3f} (p={p_dist:.2g})")

In [ ]:
true_dirs = []
dxdy  = real_pos_food - real_pos_nest
for (dx, dy) in dxdy.tolist():
    angle = (math.degrees(math.atan2(dy, dx)) + 360) % 360
    bin_idx = int((angle + 22.5) // 45) % 8
    true_dirs.append(bin_idx)

# classification accuracy of direction token
correct = sum(td == gd for td, gd in zip(token_directions, true_dirs))
acc = correct / len(true_dirs)
print(f"Direction-token accuracy = {acc*100:.1f}%  (random = {100/8:.1f}%)")

## Find meaning space in bee tokens

In [ ]:
from torch_geometric.data import Batch

aux_input = interaction_obj.aux_input
batch_data = aux_input['data']
nest_idx = aux_input['nest_idx']
food_idx = aux_input['food_idx']
messages = interaction_obj.message
vocab_size = 8

direction_tokens = messages[:, :vocab_size].argmax(dim=-1).tolist()
distance_tokens  = messages[:, -1].tolist()

all_data = []
for b in batch_data:
    if isinstance(b, Batch):
        all_data.extend(b.to_data_list())
    else:
        all_data.extend(b)

DIRECTIONS = {"N":0, "NE":1, "E":2, "SE":3, "S":4, "SW":5, "W":6, "NW":7}
ID_TO_DIR   = {v:k for k,v in DIRECTIONS.items()}

def extract_path_properties(data, nest, food):
    G = nx.DiGraph()
    edges = data.edge_index.t().tolist()
    attrs = data.edge_attr.tolist()
    for (u, v), attr in zip(edges, attrs):
        w = float(attr[0])
        raw = attr[1]
        dir_id = DIRECTIONS[raw] if isinstance(raw, str) else int(raw)
        G.add_edge(u, v, weight=w, direction=dir_id)
        G.add_edge(v, u, weight=w, direction=dir_id)

    try:
        path = nx.shortest_path(G, source=nest, target=food, weight='weight')
    except nx.NetworkXNoPath:
        return None
    if len(path) < 2:
        return None

    hop_count = len(path) - 1
    total_distance = 0.0
    directions = []
    distances  = []
    for u, v in zip(path, path[1:]):
        total_distance += G[u][v]['weight']
        distances.append(G[u][v]['weight'])
        directions.append(G[u][v]['direction'])

    direction_diversity = len(set(directions))
    direction_changes   = sum(
        1 for i in range(1, len(directions))
        if directions[i] != directions[i-1]
    )
    cnts = Counter(directions)
    dom_dir, dom_ct = cnts.most_common(1)[0]
    dom_ratio = dom_ct / len(directions)
    arr = np.array(distances)

    return {
        'hop_count': hop_count,
        'total_distance': total_distance,
        'directions': [ID_TO_DIR[d] for d in directions],
        'distances': distances,
        'direction_diversity':direction_diversity, # num unique directions in the path
        'direction_changes':  direction_changes, # num times the direction changes along the path
        'dominant_direction': ID_TO_DIR[dom_dir], # most frequently used direction in the path
        'dominant_direction_ratio': dom_ratio, # fraction of the path uses the dominant direction
        'avg_edge_distance':  arr.mean(), 
        'max_edge_distance':  arr.max(), # longest single edge in path
        'min_edge_distance':  arr.min(), # shortest single edge in path
        'distance_variance':  arr.var(), # how much edge distances vary
        'path_complexity':    direction_diversity + direction_changes
    }

records = []
for i, data in enumerate(all_data):
    ln = int(nest_idx[i].item())
    lf = int(food_idx[i].item())
    dt = direction_tokens[i]
    rt = distance_tokens[i]
    props = extract_path_properties(data, ln, lf)
    if props is None:
        continue
    rec = {
        'token_direction': ID_TO_DIR[dt],
        'token_distance':  rt
    }
    rec.update(props)
    records.append(rec)

df = pd.DataFrame(records)
df.head()

In [ ]:
df

This suggests the distance token encodes perceived navigational effort from nest node to food node rather than raw distance (how hard is this journey?). Now lets validate this if this is true.

### Distance token analysis

In [ ]:
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def plot_token_vs_effort(
        token_values: np.ndarray,
        effort_values: np.ndarray,
        axis: plt.Axes,
        effort_label: str
    ):
    # Rank-correlation (monotonic)
    spearman_rho, spearman_pval = spearmanr(token_values, effort_values)

    # linear model: token ≈ a·effort + b
    X_design          = effort_values.reshape(-1, 1)
    y_target          = token_values
    model             = LinearRegression().fit(X_design, y_target)
    token_predicted   = model.predict(X_design)
    r2_linear_fit     = r2_score(y_target, token_predicted)

    axis.scatter(effort_values, token_values, alpha=0.45, label="samples")
    effort_grid = np.linspace(effort_values.min(),
                              effort_values.max(), 200)
    axis.plot(
        effort_grid,
        model.predict(effort_grid.reshape(-1, 1)),
        color="red", lw=2,
        label=f"fit  a·x + b  (a={model.coef_[0]:.2f})"
    )
    axis.set_xlabel(effort_label)
    axis.set_ylabel("continuous token value")
    axis.set_title(f"{effort_label}\nρ={spearman_rho:.3f}, "
                   f"R²={r2_linear_fit:.3f}")
    axis.legend()

    return spearman_rho, r2_linear_fit, spearman_pval

fig, axes = plt.subplots(1, 3, figsize=(15,4))

ρ1, R1, p1 = plot_token_vs_effort(df['token_distance'].values,
                        df['total_distance'].values, axes[0], "Total distance")
ρ2, R2, p2 = plot_token_vs_effort(df['token_distance'].values,
                        df['hop_count'].astype(float).values, axes[1], "Hop count")
ρ3, R3, p3 = plot_token_vs_effort(df['token_distance'].values,
                        df['path_complexity'].astype(float).values, axes[2], "Path complexity")
plt.tight_layout(); plt.show()

print(f"Spearman ρ (token vs total_distance) = {ρ1:.3f} (p={p1:.2g})")
print(f"Spearman ρ (token vs hop_count) = {ρ2:.3f} (p={p2:.2g})")
print(f"Spearman ρ (token vs path_complexity) = {ρ3:.3f} (p={p3:.2g})")


In [ ]:
subset = df.head(5)

fig, axes = plt.subplots(1, 3, figsize=(15,4))

plot_token_vs_effort(subset['token_distance'].values,
                     subset['total_distance'].values,
                     axes[0], 'Total distance')

plot_token_vs_effort(subset['token_distance'].values,
                     subset['hop_count'].values.astype(float),
                     axes[1], 'Hop count')

plot_token_vs_effort(subset['token_distance'].values,
                     subset['path_complexity'].values.astype(float),
                     axes[2], 'Path complexity')

plt.tight_layout(); plt.show()


In [ ]:
df['token_distance'].hist(bins=8, figsize=(5,3))
plt.xlabel("token_distance value")
plt.ylabel("frequency")
plt.title("Distribution of continuous token")
plt.show()

### Direction token

In [ ]:
def build_bidirectional_digraph(data):
    graph = nx.DiGraph()
    for (src, dst), attr in zip(data.edge_index.t().tolist(), data.edge_attr.tolist()):
        edge_distance, raw_direction = attr
        if isinstance(raw_direction, str):
            dir_id = DIRECTIONS[raw_direction]
        else: 
            dir_id = int(raw_direction)

        # add both directions so graph is bidirectional
        for u, v in ((src, dst), (dst, src)):
            graph.add_edge(u, v,
                           weight   = float(edge_distance),
                           direction= dir_id)
    return graph

def bearing_sector(vec_xy):
    labels = ["N","NE","E","SE","S","SW","W","NW"]
    angle  = (math.degrees(math.atan2(vec_xy[1], vec_xy[0])) + 360) % 360
    return labels[int((angle + 22.5)//45) % 8]

records = []

for i, data in enumerate(all_data):
    nest  = int(nest_idx[i]);   food = int(food_idx[i])
    token_s = ID_TO_DIR[direction_tokens[i]]
    token_c = distance_tokens[i]

    digraph      = build_bidirectional_digraph(data)
    undirected   = nx.Graph(digraph)

    # Shortest path nest→food (weighted)
    path_nodes   = nx.shortest_path(digraph, nest, food, weight='weight')
    hop_count    = len(path_nodes) - 1
    step_dists   = []
    step_dirs    = []
    total_dist   = 0.0
    for u,v in zip(path_nodes, path_nodes[1:]):
        edge_attr   = digraph[u][v]
        total_dist += edge_attr['weight']
        step_dists.append(edge_attr['weight'])
        step_dirs .append(edge_attr['direction'])

    direction_diversity = len(set(step_dirs))
    direction_changes   = sum(d1!=d2 for d1,d2 in zip(step_dirs, step_dirs[1:])) if hop_count>1 else 0
    dom_dir, dom_ct     = Counter(step_dirs).most_common(1)[0]
    dom_ratio           = dom_ct / hop_count

    # Local connectivity features at food
    hub_node        = max(undirected.nodes, key=undirected.degree)
    try:
        hops_to_hub = nx.shortest_path_length(undirected, food, hub_node)
    except nx.NetworkXNoPath:
        hops_to_hub = np.nan
    two_hop_reach = sum(
        1 for n in undirected.nodes
        if nx.has_path(undirected, food, n)
        and nx.shortest_path_length(undirected, food, n) <= 2
    )

    import community as louvain
    partition     = louvain.best_partition(undirected)
    food_comm     = partition[food]

    # Straight-line bearing nest→food
    straight_bearing = bearing_sector((data.pos[food]-data.pos[nest]).cpu().numpy())

    records.append({
        'token_direction'           : token_s,
        'token_distance'            : token_c,
        'ground_truth_sector'       : straight_bearing,
        'hop_count'                 : hop_count,
        'total_distance'            : total_dist,
        'directions'                : [ID_TO_DIR[d] for d in step_dirs],
        'direction_diversity'       : direction_diversity,
        'direction_changes'         : direction_changes,
        'dominant_direction'        : ID_TO_DIR[dom_dir],
        'dominant_direction_ratio'  : dom_ratio,
        'avg_edge_distance'         : np.mean(step_dists),
        'max_edge_distance'         : np.max(step_dists),
        'min_edge_distance'         : np.min(step_dists),
        'distance_variance'         : np.var(step_dists),
        'path_complexity'           : direction_diversity + direction_changes,
        'hops_to_hub'               : hops_to_hub,
        'hub_degree'                : undirected.degree(hub_node),
        'two_hop_reach'             : two_hop_reach,
        'food_comm'                 : food_comm
    })

df = pd.DataFrame(records)
df.head()

In [ ]:
from scipy.stats import chi2_contingency, spearmanr, kruskal
from sklearn.metrics import mutual_info_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import community as louvain 

print("### Goal-Location Sector tests")
sector_contingency = pd.crosstab(df["token_direction"], df["ground_truth_sector"])
chi2, p_sector, _, _ = chi2_contingency(sector_contingency)
print(f"Chi-square token×sector: χ²={chi2:.1f},  p={p_sector:.3g}")

acc_sector = (df["token_direction"] == df["ground_truth_sector"]).mean()
print(f"Exact sector accuracy: {acc_sector*100:.1f} %")

le_token  = LabelEncoder().fit_transform(df["token_direction"])
le_sector = LabelEncoder().fit_transform(df["ground_truth_sector"])
mi_sector = mutual_info_score(le_token, le_sector)
print(f"Mutual information: {mi_sector:.3f} bits")

print("\nConfusion matrix (rows=actual sector, cols=token):")
print(pd.crosstab(df["ground_truth_sector"], df["token_direction"]))

# 3) ************************************************************
#    LANDMARK-BASED (hub) TESTS
# ***************************************************************
print("\n### Landmark-reference tests")
rho_hub, p_hub = spearmanr(le_token, df["hops_to_hub"], nan_policy="omit")
print(f"ρ(token , hops_to_hub)  = {rho_hub:.3f} (p={p_hub:.3g})")

median_hops = df["hops_to_hub"].median()
near_mask, far_mask = df["hops_to_hub"] <= median_hops, df["hops_to_hub"] > median_hops
H_stat, p_kw = kruskal(le_token[near_mask], le_token[far_mask])
print(f"Kruskal near vs far hubs: H={H_stat:.2f}, p={p_kw:.3g}")

# 4) ************************************************************
#    CONNECTIVITY / TOPOLOGY PROBES
# ***************************************************************
print("\n### Connectivity / topology probes")
for col in ["hub_degree", "two_hop_reach", "direction_diversity", "direction_changes"]:
    rho, p_val = spearmanr(le_token, df[col])
    print(f"ρ(token , {col}) = {rho:+.3f}  (p={p_val:.3g})")

# 5) ************************************************************
#    COMMUNITY-BASED HYPOTHESIS
# ***************************************************************
mi_comm = mutual_info_score(le_token, df["food_comm"])
print(f"\nMutual information token ↔ food-community: {mi_comm:.3f} bits")

In [ ]:
compass_labels = ["N","NE","E","SE","S","SW","W","NW"]
def id_to_label(dir_id):          return compass_labels[dir_id]
def label_to_id(label):           return compass_labels.index(label)

def dominant_label(labels):
    return Counter(labels).most_common(1)[0][0]

def net_vector_sector(dir_ids):
    x = sum(math.cos(id*math.pi/4) for id in dir_ids)
    y = sum(math.sin(id*math.pi/4) for id in dir_ids)
    return id_to_label(int(((math.degrees(math.atan2(y,x))+360)%360 + 22.5)//45)%8)

df['first_step_sector']    = df['directions'].str[0]
df['last_step_sector']     = df['directions'].str[-1]
df['dominant_step_sector'] = df['directions'].apply(dominant_label)
df['net_vector_sector']    = df['directions'].apply(
                                lambda lab: net_vector_sector([label_to_id(x) for x in lab]))

def sector_eval(sector_col):
    le_truth   = LabelEncoder().fit_transform(df[sector_col])
    le_token   = LabelEncoder().fit_transform(df['token_direction'])
    acc        = (df[sector_col] == df['token_direction']).mean()*100
    mi_bits    = mutual_info_score(le_truth, le_token)
    print(f"{sector_col:22s}  acc={acc:4.1f}%   MI={mi_bits:.3f} bits")

print("\n=== Path-based sector hypotheses ===")
for col in ['first_step_sector','last_step_sector',
            'dominant_step_sector','net_vector_sector']:
    sector_eval(col)

## Message Clustering by Referential Nodes
Test if certain nodes consistently serve as a reference point

In [ ]:
def analyze_nest_reference(interaction_obj):
    """
    For each sample, treat the nest node as the reference:
      - compute the weighted shortest‐path length from nest to food
      - extract the first‐edge direction along that path
    Then correlate those meanings against the two message tokens.
    """
    aux_input        = interaction_obj.aux_input
    batch_data       = aux_input['data']
    nest_idx         = aux_input['nest_idx']
    food_idx         = aux_input['food_idx']

    messages         = interaction_obj.message
    vocab_size       = 8
    direction_tokens = messages[:, :vocab_size].argmax(dim=-1).tolist()
    distance_tokens  = messages[:, -1].tolist()

    path_lengths = []
    first_dirs   = []

    if not isinstance(batch_data, list):
        batch_data = [batch_data]

    all_data = []
    for b in batch_data:
        all_data.extend(b.to_data_list())

    for i, data in enumerate(all_data):
        ln = int(nest_idx[i].item())
        lf = int(food_idx[i].item())
        dt =            direction_tokens[i]
        # build a bidirectional graph
        G = nx.DiGraph()
        edges = data.edge_index.t().tolist()
        attrs = data.edge_attr.tolist()
        for (u, v), attr in zip(edges, attrs):
            w = float(attr[0])
            G.add_edge(u, v, weight=w)
            G.add_edge(v, u, weight=w)

        # weighted shortest‐path length nest to food
        L = nx.shortest_path_length(G, source=ln, target=lf, weight='weight')
        path_lengths.append(L)

        # first‐edge direction along that same path
        path = nx.shortest_path(G, source=ln, target=lf, weight='weight')
        u, v  = path[0], path[1]
        # find the matching
        for (uu, vv), attr in zip(edges, attrs):
            if uu==u and vv==v:
                first_dirs.append(int(attr[1]))
                break

    # Continuous‐token vs path‐length monotonicity
    rho, pval = spearmanr(distance_tokens, path_lengths)
    print(f"Nest‐ref dist token vs. graph‐path length: Spearman rho = {rho:.3f} (p={pval:.2g})")

    # Discrete‐token vs first‐edge direction accuracy
    acc = sum(d==f for d,f in zip(direction_tokens, first_dirs)) / len(first_dirs)
    print(f"Nest‐ref direction token accuracy = {acc*100:.1f}%  (chance=12.5%)")

    return {
        'rho_dist': rho,
        'rho_pval': pval,
        'dir_acc': acc
    }


In [ ]:
analyze_nest_reference(interaction_obj)

# Human Analysis

In [ ]:
logs = load_latest_interaction(
    logs_root="../logs/interactions/2025-06-22",
    seed_folder="maxlen10_human_gs_seed42",
    split="validation",
    prefix="interaction_gpu0"
)

In [ ]:
from torch_geometric.data import Data
from typing import List

def shortest_path_distance(pyg_graph: Data, src: int, dst: int) -> torch.Tensor:
    g = nx.Graph()

    u_nodes = pyg_graph.edge_index[0].tolist()
    v_nodes = pyg_graph.edge_index[1].tolist()
    distances = pyg_graph.edge_attr[:, 1].tolist()

    for u, v, d in zip(u_nodes, v_nodes, distances):
        g.add_edge(u, v, weight=float(d))

    path_len = nx.dijkstra_path_length(g, source=src, target=dst, weight="weight")
    return torch.tensor([path_len], dtype=torch.float32)


def build_meanings_per_episode(
    batched_graphs: List[Data],
    nest_idx: torch.Tensor,
    food_idx: torch.Tensor,
) -> torch.Tensor:
    meaning_vectors: List[torch.Tensor] = []
    global_id = 0 

    for batch in batched_graphs: 
        episodes = batch.to_data_list()
        for local_ep, episode in enumerate(episodes):
            src = nest_idx[global_id].item()
            dst = food_idx[global_id].item() 
            meaning_vec = shortest_path_distance(episode, src, dst)
            meaning_vectors.append(meaning_vec)
            global_id += 1

    return torch.vstack(meaning_vectors)  

In [ ]:
human_messages

In [ ]:
batches  = logs.aux_input["data"]
nests    = logs.aux_input["nest_idx"]
foods    = logs.aux_input["food_idx"]

meanings = build_meanings_per_episode(batches, nests, foods)

human_messages = [m.tolist() for m in logs.message.argmax(dim=-1)]

topsim = TopographicSimilarity.compute_topsim(
    meanings=meanings,
    messages=human_messages,
    meaning_distance_fn="euclidean",
    message_distance_fn="edit"
)
print(topsim)


In [ ]:
human_messages

# TopSim score over epochs

In [ ]:
all_logs = load_all_interactions(
    logs_root="../logs/interactions",
    seed_folder="human_gs_seed42",
    split="validation",
    prefix="interaction_gpu0"
)

In [ ]:
topsim_scores_over_epochs = []
pos_topsim_scores_over_epochs = []

for i, logs in enumerate(all_logs):
    print(f"Processing Epoch {i}...")

    human_messages = logs.message.argmax(dim=-1)
    human_messages_list = [msg.tolist() for msg in human_messages]

    meanings = torch.stack([
        logs.aux_input["nest_tensor"],
        logs.aux_input["food_tensor"]
    ], dim=1)

    meanings_pos = torch.cat([
        logs.aux_input['nest_pos'],
        logs.aux_input['food_pos']
    ], dim=1)

    topsim = TopographicSimilarity.compute_topsim(
        meanings=meanings,
        messages=human_messages_list,
        meaning_distance_fn="hamming",
        message_distance_fn="edit"
    )

    pos_topsim = TopographicSimilarity.compute_topsim(
        meanings=meanings_pos,
        messages=human_messages_list,
        meaning_distance_fn="euclidean",
        message_distance_fn="edit"
    )

    topsim_scores_over_epochs.append(topsim)
    pos_topsim_scores_over_epochs.append(pos_topsim)

print("\nAll epochs processed.")
print("Symbolic TopSim scores:", topsim_scores_over_epochs)
print("Positional TopSim scores:", pos_topsim_scores_over_epochs)

In [ ]:
epochs = range(len(topsim_scores_over_epochs))

plt.figure(figsize=(12, 7))

plt.plot(epochs, topsim_scores_over_epochs, linestyle='-', label='Symbolic TopSim (Hamming)')
plt.plot(epochs, pos_topsim_scores_over_epochs, linestyle='-', label='Positional TopSim (Euclidean)')

plt.title('TopSim over Epochs')
plt.xlabel('Epoch')
plt.ylabel('TopSim Score')
plt.legend()

plt.show()

In [ ]:
import editdistance
logs_root = "../logs/interactions/2025-06-16"
seed_folders = [
    "gamesize10_bee_gs_seed42",
    "gamesize10_bee_gs_seed123",
    "gamesize10_bee_gs_seed2025",
]

scores = []
for seed in seed_folders:
    logs = load_latest_interaction(
        logs_root=logs_root,
        seed_folder=seed,
        split="validation",
        prefix="interaction_gpu0"
    )

    messages = logs.message.argmax(dim=-1).cpu().tolist()

    nest = np.stack(logs.aux_input["nest_pos"])
    food = np.stack(logs.aux_input["food_pos"])
    delta = food - nest
    dist = np.linalg.norm(delta, axis=1)
    theta = np.arctan2(delta[:,1], delta[:,0])
    meanings = np.stack([dist, theta], axis=1)

    ts = TopographicSimilarity.compute_topsim(
        meanings=meanings,
        messages=messages,
        meaning_distance_fn="euclidean",
        message_distance_fn=lambda x, y: editdistance.eval(x, y) / ((len(x) + len(y)) / 2)
    )
    scores.append(ts)

mean_score = np.mean(scores)
std_score  = np.std(scores)

fig, ax = plt.subplots()
ax.bar(['Human'], [mean_score], yerr=[std_score], capsize=5)
ax.set_ylabel('TopSim Score')
ax.set_title('Mean TopSim across seeds')
plt.show()

In [ ]:
import numpy as np
import editdistance
from scipy.optimize import linear_sum_assignment
from collections import Counter, defaultdict

def compute_cbm(messages, dist_bins, dir_bins, n_dist_bins, n_dir_bins):
    C = n_dist_bins + n_dir_bins
    Q = defaultdict(int)
    for i, msg in enumerate(messages):
        # map dir concepts to IDs n_dist_bins…n_dist_bins+n_dir_bins-1
        concepts = [ dist_bins[i], n_dist_bins + dir_bins[i] ]
        for w in msg:
            for v in concepts:
                Q[(w,v)] += 1

    W = max(w for msg in messages for w in msg) + 1
    mat = np.zeros((W, C), int)
    for (w,v), cnt in Q.items():
        mat[w,v] = cnt

    row, col = linear_sum_assignment(-mat)
    match_mass = mat[row, col].sum()
    norm = sum(max(len(msg), 2) for msg in messages)
    return match_mass / norm

dx = food[:,0] - nest[:,0]
dy = food[:,1] - nest[:,1]
distances = np.hypot(dx, dy)  
dirs = ["N","NE","E","SE","S","SW","W","NW"]
angles = (np.degrees(np.arctan2(dy, dx)) % 360)
dir_bins = np.array([
    int((angle + 22.5)//45) % 8
    for angle in angles
])
n_dist_bins = 1
quantiles = np.linspace(0, 1, n_dist_bins+1)
edges = np.quantile(distances, quantiles)
dist_bins = np.digitize(distances, edges[1:-1])

cbm = compute_cbm(
    messages=human_messages,
    dist_bins=dist_bins,
    dir_bins=dir_bins,
    n_dist_bins = n_dist_bins,
    n_dir_bins  = 8
)
print(f"Concept‐Best‐Matching score = {cbm:.4f}")
